# Adaptive Partition-Based Level Decomposition Methods for Solving Two-Stage Stochastic Programs with Fixed Recourse
[W. van Ackooij](mailto:wim.van.ackooij@gmail.com), [W. de Oliveira](mailto:welington@ime.uerj.br) and [Yongjia Song](mailto:yongjis@clemson.edu)

In [1]:
# techincal lines
using LinearAlgebra
using JuMP
using Gurobi

#create gurobi environment
const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only - expires 2021-05-08


In [2]:
#reading the data file
include("data.jl");

In [3]:
#define the model
MVP = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));

#define the variables
@variable(MVP,u[1:nbFirstVars]); #u is the 1st stage variable in the MVP
@variable(MVP,v[1:nbSecVars]);   #v is the 2nd stage variable in the MVP


#define the objective
@objective(MVP,Min,sum(objcoef[i]*u[i] for i=1:nbFirstVars)+sum(objcoef[nbFirstVars+j]*v[j] for j=1:nbSecVars));


#define the constraints

#the 1st satge variable is bigger/smaller than its lower/upper bound
for i=1:nbFirstVars
    @constraint(MVP, u[i] >=firstvarlb[i]);
    @constraint(MVP, u[i] <=firstvarub[i]);
end

#the 2nd satge variable is bigger/smaller than the expected value of its lower/upper bound
for i=1:nbSecVars
    @constraint(MVP, v[i] >= (sum(secondvarlb[k,i] for k=1:nbScens)/nbScens));
    @constraint(MVP, v[i] <= (sum(secondvarub[k,i] for k=1:nbScens)/nbScens));
end

#the lower/upper bound constraints related to the 1st satge variable 
for i=1:nbFirstRows
    @constraint(MVP, sum(u[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) >=firstconstrlb[i])    
    @constraint(MVP, sum(u[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) <=firstconstrub[i])    
end

#the bound constraints related to the 2nd satge variable 
for i=1:nbSecRows
    if secondconstrsense[i] == 0
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) == (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
    if secondconstrsense[i] == 1
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) >= (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
    if secondconstrsense[i] == -1
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) <= (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
end

In [4]:
optimize!(MVP)
status_mvp = termination_status(MVP);
if status_mvp != MOI.OPTIMAL
    println("Models status is: ", status_mvp, " === Oops! :/")
else
    obj_mvp = objective_value(MVP);
    xval = value.(u);
    yval = value.(v)
    println("Finish solving! Status = ", status_mvp);
    println("Optimal objective value = ", obj_mvp);
    println("x optimal value = ", xval);
    println("y optimal value = ", yval);
end

Finish solving! Status = OPTIMAL
Optimal objective value = 218.01398122003295
x optimal value = [0.0, 4.382227853768909, 0.0, 2.143650429190988, 0.15390412269472595, 0.0, 0.00907823792055301, 0.0, 0.4076869778625408, 0.0, 0.010530876431487727, 0.8259507917591504, 0.0, 0.0, 0.0, 0.07554298065344937, 0.1124766053641693, 0.29335021017563373, 0.1896740640303116, 0.13928779971105198]
y optimal value = [0.3701034309592, 0.4134238912422419, 2.245219202035589, 0.0, 0.0, 0.27798970024234154, 1.9257370492861778, 1.9669669988627463, 8.706661809364638, 2.872718146711897, 0.8105021826000701, 0.0, 2.8396558031285073, 0.0, 0.27609095167001063, 0.0, 2.2041596495119946, 0.17142382332988335, 0.0, 3.0010287475706714, 2.7729648641220646, 2.816580851704152, 2.11346308726969, 0.0, 0.0, 0.0, 7.054344486774584, 0.0, 0.0, 0.0]


In [5]:
#coarse cut
function SPC(xval,P,p)
    piC = zeros(p,nbSecRows+nbSecVars*2);
    QC = zeros(p);
    
    #For each cluster we will do the following 
    for k=1:p
        for i=1:nbSecVars
            #update the 2nd stage variable LB & UB
            TempRHSlb = sum(secondvarlb[j,i] for j in P[k])/length(P[k])
            set_normalized_rhs(constrlb[i], TempRHSlb);
            
            TempRHSub = sum(secondvarub[j,i] for j in P[k])/length(P[k])
            set_normalized_rhs(construb[i], TempRHSub);
        end
        
        #update the bound of each of the constraints 
        for i=1:nbSecRows
            if secondconstrsense[i] == 0
                TempRHS1 = sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]));
                set_normalized_rhs(constr1[i], TempRHS1);
            end
            
            if secondconstrsense[i] == 1
                TempRHS2 = sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]));
                set_normalized_rhs(constr2[i], TempRHS2);
            end
            if secondconstrsense[i] == -1
                TempRHS3 = sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]));
                set_normalized_rhs(constr3[i], TempRHS3);
            end
        end
        #We solve the subproblem & obtain its objective value
        optimize!(SubProblem)
        statusSP = termination_status(SubProblem);
        if statusSP != MOI.OPTIMAL
            println("Models status is: ", statusSP, " === Oops! :/")
            exit()
        else
            QC[k] = objective_value(SubProblem);
            
            #We get the dual multiplier of each constraint for this scenario 
            for i=1:nbSecVars
                if -1.0e10<secondvarlb[i]
                    piC[k,i] = dual(constrlb[i])
                end
                if 1.0e10>secondvarub[i]
                    piC[k,nbSecVars+i] = dual(construb[i])
                end
            end
            for i=1:nbSecRows
                if secondconstrsense[i] == 0
                    piC[k,nbSecVars+nbSecVars+i]=dual(constr1[i]);
                end
                if secondconstrsense[i] == 1
                    piC[k,nbSecVars+nbSecVars+i]=dual(constr2[i]);
                end
                if secondconstrsense[i] == -1
                    piC[k,nbSecVars+nbSecVars+i]=dual(constr3[i]);
                end  
            end
        end
    end
    return QC, piC
end

SPC (generic function with 1 method)

In [6]:
#coarse cut
function SPSC(xval,P,c)
    
    #For each scenario in the cluster we will do the following 
    piSC = zeros(nbScens,nbSecRows+nbSecVars*2);
    QSC = zeros(nbScens);
    #For each scenario we will do the following 
    for k in P[c]
        for i=1:nbSecVars
            #update the 2nd stage variable LB & UB
            TempRHSlb = secondvarlb[k,i]
            set_normalized_rhs(constrlb[i], TempRHSlb);
            TempRHSub = secondvarub[k,i]
            set_normalized_rhs(construb[i], TempRHSub);
        end
        
        #update the bound of each of the constraints 
        for i=1:nbSecRows
            if secondconstrsense[i] == 0
                TempRHS1 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr1[i], TempRHS1);
            end
            if secondconstrsense[i] == 1
                TempRHS2 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr2[i], TempRHS2);
            end
            if secondconstrsense[i] == -1
                TempRHS3 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr3[i], TempRHS3);
            end
        end
        #We solve the subproblen & obtain its objective value
        optimize!(SubProblem)
        statusSP = termination_status(SubProblem);
        if statusSP != MOI.OPTIMAL
            println("Models status is: ", statusSP, " === Oops! :/")
            #exit()
        else
            QSC[k] = objective_value(SubProblem);
            #We get the dual multiplier of each constraint for this scenario 
            for i=1:nbSecVars
                piSC[k,i] = dual(constrlb[i])
                piSC[k,nbSecVars+i] = dual(construb[i])
            end
            for i=1:nbSecRows
                if secondconstrsense[i] == 0
                    piSC[k,nbSecVars+nbSecVars+i]=dual(constr1[i]);
                end
                if secondconstrsense[i] == 1
                    piSC[k,nbSecVars+nbSecVars+i]=dual(constr2[i]);
                end
                if secondconstrsense[i] == -1
                    piSC[k,nbSecVars+nbSecVars+i]=dual(constr3[i]);
                end  
            end
            
        end

    end
    return QSC, piSC
end

SPSC (generic function with 1 method)

In [7]:
function solve_master(LB,xval,thetaval)
    optimize!(Master)
    status_master = termination_status(Master);
    if status_master != MOI.OPTIMAL
        println("Models status is: ", status_master, " === Oops! :/")
        exit()
    else
        LB = objective_value(Master);
        xval = value.(x);
        thetaval = value(theta);
    end
    return LB, xval, thetaval
end

solve_master (generic function with 1 method)

In [8]:
 function Refine(P,PC,g,pi,ϵ,xxval,Q)
    Refine_Start=time();
    temp = [];
    reps = [];
    reps_indc = [];
    push!(temp,[PC[g][1]])
    push!(reps,pi[PC[g][1],:]);
    push!(reps_indc,PC[g][1]);
    
    for k=2:length(PC[g])
        count = 0;
        for s=1:length(temp)
            diff = pi[PC[g][k],:]-reps[s];
            if norm(diff) < ϵ 
                push!(temp[s],PC[g][k])
                break
            else
                #check if the solution is degenerate solutions
                obj_diff = Q[PC[g][k]] - Q[reps_indc[s]]
                if abs(obj_diff)/abs(Q[reps_indc[s]]) < ϵ
                    push!(temp[s],PC[g][k])
                    break
                else
                    count +=1
                end
            end
        end
        if count == length(temp)
            push!(temp,[PC[g][k]])
            push!(reps, pi[PC[g][k],:]);
            push!(reps_indc, PC[g][k]);
        end
    end 
    
    filter!(e->e!=PC[g],P)
    for i=1:length(temp)
        push!(P,temp[i])
    end
    
    elaps = time()-Refine_Start;
    return P, elaps
end

Refine (generic function with 1 method)

In [9]:
#define the master problem
Master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
@variable(Master,x[1:nbFirstVars]);
@variable(Master,theta);
@objective(Master,Min,sum(objcoef[i]*x[i] for i=1:nbFirstVars)+theta);

#To prevent our Master problem solution in the first iteration from being unbounded since we don't have a bound on theta, we add a constraint were our objective is bounded by the MVP objective 
@constraint(Master, sum(objcoef[i]*x[i] for i=1:nbFirstVars)+theta>=getobjectivevalue(MVP));

#lower/upper bound on the 1st stage variable 
for i=1:nbFirstVars
    if -1.0e10<firstvarlb[i]
        @constraint(Master, x[i] >=firstvarlb[i]);
    end
    if 1.0e10>firstvarub[i]
        @constraint(Master, x[i] <=firstvarub[i]);
    end
end

#the lower/upper bound constraints related to the 1st satge variable 
for i=1:nbFirstRows
    @constraint(Master, sum(x[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) >=firstconstrlb[i]);    
    @constraint(Master, sum(x[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) <=firstconstrub[i]);    
end

In [10]:
# define the second stage subproblem
SubProblem = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
@variable(SubProblem,y[1:nbSecVars]);
@objective(SubProblem,Min,sum(objcoef[nbFirstVars+j]*y[j] for j=1:nbSecVars));

#We will initilaize the RHS of all the 2nd stage constraints to be zero and we will change it later
@constraint(SubProblem,constrlb[i=1:nbSecVars], y[i] >= 0);
@constraint(SubProblem,construb[i=1:nbSecVars], y[i] <= 0);
@constraint(SubProblem,constr1[i=1:nbSecRows; secondconstrsense[i] == 0], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) == 0);
@constraint(SubProblem,constr2[i=1:nbSecRows; secondconstrsense[i] == 1], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) >= 0);
@constraint(SubProblem,constr3[i=1:nbSecRows; secondconstrsense[i] == -1], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) <= 0);

In [11]:
#initialize stuff
P = [[k for k=1:nbScens]]; #The initial partition
PC = deepcopy(P); #a copy of the partition
p = length(P); 

LB = obj_mvp;
UB = 1e10;

ϵ = 1e-5;
iter = 0;

thetaval = -1e10

start=time();
while (UB-LB)*1.0/max(1e-10,abs(UB)) > 1e-4
    iter = iter + 1;
    LB, xval, thetaval = solve_master(LB,xval,thetaval);
    
    #attempt to generate a coarse cut
    QC, piC = SPC(xval,P,p);
    θtemp = sum(QC[s]*(length(P[s])/nbScens) for s=1:p);    
    if length(PC) == nbScens
        UB = min(LB-thetaval+θtemp,UB);
    end
    
    if (θtemp-thetaval)/max(1e-10,abs(thetaval)) > ϵ
        @constraint(Master,
                    theta>= 
                    sum((sum(piC[k,i]*sum(secondvarlb[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                   +sum(piC[k,nbSecVars+i]*sum(secondvarub[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                   +sum(piC[k,nbSecVars+nbSecVars+i]*(sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) 
                   -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows))*(length(P[k])/nbScens) for k=1:p));
    else
        #generate a semi_coarse cut
        
        tempUB = 0;
        for c=1:length(PC)
            QSC, piSC = SPSC(xval,P,c);
            θtemp = sum(QSC[k] for k in PC[c])*(1/nbScens)
            tempUB += θtemp; 
            if length(PC) > 1
                θtemp += sum(QC[k]*(length(PC[k])/nbScens) for k=1:p if k != c);
            end
            if c == length(PC)
                UB = min(LB-thetaval+tempUB,UB);
            end

            
            if (θtemp-thetaval)/max(1e-10,abs(thetaval)) > ϵ
                if p == length(PC)
                    @constraint(Master,
                        theta>= 
                        sum(sum(piSC[k,i]*secondvarlb[k,i] for i=1:nbSecVars)
                       +sum(piSC[k,nbSecVars+i]*secondvarub[k,i] for i=1:nbSecVars)
                       +sum(piSC[k,nbSecVars+nbSecVars+i]*(secondconstrbd[k,i] 
                       -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows) for k in P[c])*(1/nbScens)
                                );
                else
                    @constraint(Master,
                        theta>= 
                        sum(sum(piSC[k,i]*secondvarlb[k,i] for i=1:nbSecVars)
                       +sum(piSC[k,nbSecVars+i]*secondvarub[k,i] for i=1:nbSecVars)
                       +sum(piSC[k,nbSecVars+nbSecVars+i]*(secondconstrbd[k,i] 
                       -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows) for k in P[c])*(1/nbScens)
                       +sum((sum(piC[k,i]*sum(secondvarlb[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                       +sum(piC[k,nbSecVars+i]*sum(secondvarub[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                       +sum(piC[k,nbSecVars+nbSecVars+i]*(sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) 
                       -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows))*(length(P[k])/nbScens) for k=1:p if k != c)         
                                );
                end

                if length(PC[c]) > 1
                    #P, elaps = Refine(P,PC,c,piSC,ϵ,xval,QSC);
                    P, elaps = Refine(P,PC,c,piSC,1e-15,xval,QSC);
                end
                P = sort!(P, by=length, rev=true);
                PC = deepcopy(P);
                p = length(P);
                break
            end
        end
        
    end
    #given the new dual multiplier, refine the partition
    #check to make sure # of clusters < the total # of scenarios
    println("iter = ", iter);
    println("LB = ", LB);
    println("UB = ", UB);
    println("===============================================================");
end
Elapsed = time()-start;
println("******************************************")
println("******************************************")
println("Problem is solved! Yaay")
println("******************************************")
println("Elapsed time = ", Elapsed, " seconds")
println("iter = ", iter);
println("# of clusters used  = ", p);
println("LB = ", LB);
println("UB = ", UB);
println("xvalue = ", xval);

iter = 1
LB = 218.01398122003295
UB = 1.0e10
iter = 2
LB = 218.01398122003297
UB = 1.0e10
iter = 3
LB = 218.0139812200329
UB = 1.0e10
iter = 4
LB = 218.01398122003295
UB = 1.0e10
iter = 5
LB = 218.01398122003295
UB = 1.0e10
iter = 6
LB = 218.0139812200329
UB = 272.4830269062536
iter = 7
LB = 233.90075087479704
UB = 272.4830269062536
iter = 8
LB = 234.60764602734778
UB = 272.4830269062536
iter = 9
LB = 235.7553988815227
UB = 272.4830269062536
iter = 10
LB = 237.26411997119612
UB = 272.4830269062536
iter = 11
LB = 237.34139124165736
UB = 272.4830269062536
iter = 12
LB = 237.34761616435296
UB = 272.4830269062536
iter = 13
LB = 237.3669365089479
UB = 272.4830269062536
iter = 14
LB = 237.3669365089479
UB = 272.4830269062536
iter = 15
LB = 241.73420420516396
UB = 272.4830269062536
iter = 16
LB = 242.59188437335953
UB = 272.4830269062536
iter = 17
LB = 243.25155725955753
UB = 272.4830269062536
iter = 18
LB = 243.77314109085825
UB = 272.4830269062536
iter = 19
LB = 243.78158110447436
UB = 272.

iter = 70
LB = 247.916293354707
UB = 272.4830269062536
iter = 71
LB = 247.916293354707
UB = 272.4830269062536
iter = 72
LB = 247.92670182111343
UB = 272.4830269062536
iter = 73
LB = 247.9548433280507
UB = 272.4830269062536
iter = 74
LB = 247.9579921420996
UB = 272.4830269062536
iter = 75
LB = 247.95826046899984
UB = 272.4830269062536
iter = 76
LB = 247.95826046899984
UB = 272.4830269062536
iter = 77
LB = 248.06103370215877
UB = 272.4830269062536
iter = 78
LB = 248.21570359838194
UB = 272.4830269062536
iter = 79
LB = 248.27141347494327
UB = 272.4830269062536
iter = 80
LB = 248.3086488923637
UB = 272.4830269062536
iter = 81
LB = 248.33979555989998
UB = 272.4830269062536
iter = 82
LB = 248.34050308453615
UB = 272.4830269062536
iter = 83
LB = 248.34050308453615
UB = 272.4830269062536
iter = 84
LB = 248.47186452094383
UB = 272.4830269062536
iter = 85
LB = 248.6314078265322
UB = 272.4830269062536
iter = 86
LB = 248.66756390899357
UB = 272.4830269062536
iter = 87
LB = 248.67353936757098
UB = 

iter = 138
LB = 249.84614054114385
UB = 272.4830269062536
iter = 139
LB = 249.84614054114385
UB = 272.4830269062536
iter = 140
LB = 249.91843568163216
UB = 272.4830269062536
iter = 141
LB = 249.92007363698602
UB = 272.4830269062536
iter = 142
LB = 249.96250147000984
UB = 272.4830269062536
iter = 143
LB = 249.96291856095618
UB = 272.4830269062536
iter = 144
LB = 249.963530964217
UB = 272.4830269062536
iter = 145
LB = 249.9643290241124
UB = 272.4830269062536
iter = 146
LB = 249.96674482553476
UB = 272.4830269062536
iter = 147
LB = 249.97306637383628
UB = 272.4830269062536
iter = 148
LB = 249.97451363363666
UB = 272.4830269062536
iter = 149
LB = 249.97451363363666
UB = 272.4830269062536
iter = 150
LB = 249.97676385484462
UB = 272.4830269062536
iter = 151
LB = 249.97715625687076
UB = 272.4830269062536
iter = 152
LB = 249.97804217805714
UB = 272.4830269062536
iter = 153
LB = 249.97804217805714
UB = 272.4830269062536
iter = 154
LB = 250.00772272524247
UB = 272.4830269062536
iter = 155
LB = 2

iter = 206
LB = 250.13159419499382
UB = 272.4830269062536
iter = 207
LB = 250.13159419499382
UB = 272.4830269062536
iter = 208
LB = 250.13270642657233
UB = 272.4830269062536
iter = 209
LB = 250.13270642657233
UB = 272.4830269062536
iter = 210
LB = 250.13338840448492
UB = 272.4830269062536
iter = 211
LB = 250.13338840448492
UB = 272.4830269062536
iter = 212
LB = 250.13436249603618
UB = 272.4830269062536
iter = 213
LB = 250.13485833251283
UB = 272.4830269062536
iter = 214
LB = 250.13485833251283
UB = 272.4830269062536
iter = 215
LB = 250.13733746753914
UB = 272.4830269062536
iter = 216
LB = 250.1387465133991
UB = 272.4830269062536
iter = 217
LB = 250.14085008731624
UB = 272.4830269062536
iter = 218
LB = 250.14273370344404
UB = 272.4830269062536
iter = 219
LB = 250.1427612950006
UB = 272.4830269062536
iter = 220
LB = 250.1427612950006
UB = 272.4830269062536
iter = 221
LB = 250.14473524962213
UB = 272.4830269062536
iter = 222
LB = 250.14534415119505
UB = 272.4830269062536
iter = 223
LB = 2

iter = 274
LB = 250.3280600686146
UB = 272.4830269062536
iter = 275
LB = 250.33631150218307
UB = 272.4830269062536
iter = 276
LB = 250.33777538805077
UB = 272.4830269062536
iter = 277
LB = 250.33942415639103
UB = 272.4830269062536
iter = 278
LB = 250.34177935909022
UB = 272.4830269062536
iter = 279
LB = 250.34208353376735
UB = 272.4830269062536
iter = 280
LB = 250.34238250411764
UB = 272.4830269062536
iter = 281
LB = 250.34279942842497
UB = 272.4830269062536
iter = 282
LB = 250.34279942842497
UB = 272.4830269062536
iter = 283
LB = 250.34369373027786
UB = 272.4830269062536
iter = 284
LB = 250.34488142616755
UB = 272.4830269062536
iter = 285
LB = 250.3471887579217
UB = 272.4830269062536
iter = 286
LB = 250.3471887579217
UB = 272.4830269062536
iter = 287
LB = 250.35220876356743
UB = 272.4830269062536
iter = 288
LB = 250.35696817230723
UB = 272.4830269062536
iter = 289
LB = 250.35788883808735
UB = 272.4830269062536
iter = 290
LB = 250.3599028891064
UB = 272.4830269062536
iter = 291
LB = 25

iter = 342
LB = 250.4417109460534
UB = 272.4830269062536
iter = 343
LB = 250.44262417586788
UB = 272.4830269062536
iter = 344
LB = 250.452340641424
UB = 272.4830269062536
iter = 345
LB = 250.4559008333637
UB = 272.4830269062536
iter = 346
LB = 250.4597040041727
UB = 272.4830269062536
iter = 347
LB = 250.46298022126734
UB = 272.4830269062536
iter = 348
LB = 250.46322016934843
UB = 272.4830269062536
iter = 349
LB = 250.46322016934843
UB = 272.4830269062536
iter = 350
LB = 250.46546176683927
UB = 272.4830269062536
iter = 351
LB = 250.46549220611487
UB = 272.4830269062536
iter = 352
LB = 250.46551244261624
UB = 272.4830269062536
iter = 353
LB = 250.46551244261624
UB = 272.4830269062536
iter = 354
LB = 250.47003423476264
UB = 272.4830269062536
iter = 355
LB = 250.47053926563518
UB = 272.4830269062536
iter = 356
LB = 250.4716086648443
UB = 272.4830269062536
iter = 357
LB = 250.47169795166147
UB = 272.4830269062536
iter = 358
LB = 250.47257317669863
UB = 272.4830269062536
iter = 359
LB = 250.

iter = 410
LB = 250.5364103603436
UB = 272.4830269062536
iter = 411
LB = 250.53664898297023
UB = 272.4830269062536
iter = 412
LB = 250.53664898297026
UB = 272.4830269062536
iter = 413
LB = 250.5368618084951
UB = 272.4830269062536
iter = 414
LB = 250.53686180849516
UB = 272.4830269062536
iter = 415
LB = 250.53980724723613
UB = 272.4830269062536
iter = 416
LB = 250.54005971024165
UB = 272.4830269062536
iter = 417
LB = 250.54072887597695
UB = 272.4830269062536
iter = 418
LB = 250.54072887597695
UB = 272.4830269062536
iter = 419
LB = 250.54251850934847
UB = 272.4830269062536
iter = 420
LB = 250.54271980508096
UB = 272.4830269062536
iter = 421
LB = 250.54284110882017
UB = 272.4830269062536
iter = 422
LB = 250.54284110882017
UB = 272.4830269062536
iter = 423
LB = 250.5440772533691
UB = 272.4830269062536
iter = 424
LB = 250.54469270073054
UB = 272.4830269062536
iter = 425
LB = 250.54574381243398
UB = 272.4830269062536
iter = 426
LB = 250.54574381243398
UB = 272.4830269062536
iter = 427
LB = 2